In [14]:
import torch
import torch.nn.functional as F

# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Parameters
gamma = 1e-8
beta = 1e8
lambda_init = 1e-4
n_iterations = 10**6
burn_in_samples = 10000
q_bar = 0.95
n_samples = 10000

def compute_softmax(w):
    return F.softmax(w - torch.max(w), dim=0)

def generate_samples(dist_params, n_samples):
    return torch.stack([
        torch.normal(mean=mu, std=sigma, size=(n_samples,)).to(device)
        for mu, sigma in dist_params
    ])

# --- SGLD CVaR Minimization (with standardization) ---
def sgld_cvar(x_samples):
    # Standardize
    means = torch.mean(x_samples, dim=1, keepdim=True)
    stds = torch.std(x_samples, dim=1, keepdim=True) + 1e-8
    x_std = (x_samples - means) / stds

    # Initialize theta from quantile
    initial_losses = -torch.sum(x_std, dim=0)
    theta = torch.tensor(torch.quantile(initial_losses, q_bar).item(), device=device, requires_grad=True)
    weights = torch.randn(3, device=device, requires_grad=True)

    optimizer = torch.optim.SGD([theta, weights], lr=lambda_init)
    theta_values = []

    for i in range(n_iterations):
        idx = torch.randint(0, n_samples, (1,))
        x_sample = x_std[:, idx].squeeze(1)
        softmax_weights = compute_softmax(weights)
        portfolio_loss = -torch.sum(softmax_weights * x_sample)

        loss = theta + (1 / (1 - q_bar)) * F.relu(portfolio_loss - theta) \
               + gamma * torch.sum(weights**2) + gamma * theta**2

        noise_theta = torch.randn(1, device=device) * torch.sqrt(torch.tensor(2 * lambda_init / beta))
        noise_weights = torch.randn(3, device=device) * torch.sqrt(torch.tensor(2 * lambda_init / beta))

        optimizer.zero_grad()
        loss.backward()

        with torch.no_grad():
            theta -= lambda_init * theta.grad + noise_theta.item()
            weights -= lambda_init * weights.grad + noise_weights
            theta.grad.zero_()
            weights.grad.zero_()

        if i >= n_iterations - burn_in_samples:
            theta_values.append(theta.item())

    final_weights = compute_softmax(weights).detach().cpu().numpy()
    losses = [-torch.sum(compute_softmax(weights).detach() * x_std[:, i]).item() for i in range(n_samples)]
    var = torch.mean(torch.tensor(theta_values)).item()
    cvar = var + torch.mean(F.relu(torch.tensor(losses) - var)) / (1 - q_bar)
    return var, cvar.item(), final_weights  # negative sign = back to return space

# --- Grid Search Reference (also standardized) ---
def grid_search_ref(x_samples):
    means = torch.mean(x_samples, dim=1, keepdim=True)
    stds = torch.std(x_samples, dim=1, keepdim=True) + 1e-8
    x_std = (x_samples - means) / stds

    grid = torch.linspace(0, 1, 100)
    best_cvar = float("inf")
    best_var = None
    best_weights = None

    for w1 in grid:
        for w2 in grid:
            if w1 + w2 > 1:
                continue
            w3 = 1 - w1 - w2
            weights = torch.tensor([w1, w2, w3], device=device)
            losses = -torch.sum(weights.unsqueeze(1) * x_std, dim=0)
            var = torch.quantile(losses, q_bar).item()
            cvar = var + torch.mean(F.relu(losses - var)) / (1 - q_bar)
            if cvar < best_cvar:
                best_cvar = cvar.item()
                best_var = var
                best_weights = weights.cpu().numpy()

    return best_var, best_cvar, best_weights

# --- Run All Test Cases ---
def main():
    torch.manual_seed(42)
    cases = [
        [(500, 1), (0, 1e6), (0, 1e-4)],
        [(500, 1), (0, 1e6), (0, 1)],
        [(0, 1e3), (0, 1), (0, 4)],
        [(0, 1), (1, 4), (0, 1e-4)],
        [(0, 1), (1, 4), (2, 1)]
    ]

    print(f"{'Case':<6} {'w1':>6} {'w2':>6} {'w3':>6} {'VaR_SGLD':>10} {'CVaR_SGLD':>12} {'w1*':>6} {'w2*':>6} {'w3*':>6} {'VaR*':>10} {'CVaR*':>10}")
    print("-" * 95)

    for idx, dist_params in enumerate(cases):
        x_samples = generate_samples(dist_params, n_samples)
        var_sgld, cvar_sgld, weights_sgld = sgld_cvar(x_samples)
        var_ref, cvar_ref, weights_ref = grid_search_ref(x_samples)

        print(f"{idx+1:<6} "
              f"{weights_sgld[0]:6.3f} {weights_sgld[1]:6.3f} {weights_sgld[2]:6.3f} "
              f"{var_sgld:10.3f} {cvar_sgld:12.3f} "
              f"{weights_ref[0]:6.3f} {weights_ref[1]:6.3f} {weights_ref[2]:6.3f} "
              f"{var_ref:10.3f} {cvar_ref:10.3f}")

if __name__ == "__main__":
    main()


Case       w1     w2     w3   VaR_SGLD    CVaR_SGLD    w1*    w2*    w3*       VaR*      CVaR*
-----------------------------------------------------------------------------------------------
1       0.332  0.341  0.327      0.935        1.165  0.333  0.343  0.323      0.942      1.165
2       0.325  0.358  0.317      0.962        1.211  0.333  0.343  0.323      0.956      1.210
3       0.333  0.356  0.310      0.942        1.201  0.323  0.354  0.323      0.953      1.201
4       0.333  0.342  0.324      0.934        1.172  0.333  0.343  0.323      0.938      1.172
5       0.327  0.323  0.351      0.925        1.166  0.333  0.323  0.343      0.934      1.166
